# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.model_selection import StratifiedKFold

# Loading the data

In [2]:
data_df = pd.read_csv('~/Downloads/Resistance fraction mRNA and RECIST.csv')
data_df

,SAMPLE_ID,Study,ID,Dose,mRECIST,FR,ABCB1,ABCC3,ABCC10,ABCG2,...,VPS37A,WDR12,WEE1,WNT11,WT1,WWTR1,XRCC1,YAP1,ZEB1,ZRSR2
0,PDX060,555_17,21,100,PD,0.927323,0.000000,2.214006,2.332022,0.029046,...,2.461488,3.358016,2.856228,1.538509,0.064439,3.150992,1.943299,3.822867,0.936241,1.334747
1,PDX094,576_17,685,100,PD,0.956410,0.000000,1.294543,3.228891,0.054690,...,2.658678,3.107723,3.073935,1.218616,0.871460,3.101913,2.098261,3.186725,0.737357,1.994699
2,PDX102,642_17,1613,100,PD,0.949115,0.000000,2.769161,2.833882,0.231349,...,3.060974,3.128222,3.557981,2.276709,0.402820,3.430453,2.533307,3.584706,0.741203,2.595618
3,PDX124,587_17,882,100,SD,0.334895,0.000000,0.927595,1.538783,0.221900,...,2.421085,3.309314,2.455633,2.120032,0.139922,2.903910,2.431486,3.221850,0.338023,2.138889
4,PDX127,558_17,269,100,PD,0.908467,0.042825,1.774764,2.542196,0.027021,...,2.629752,3.833862,3.436647,0.547888,0.356963,3.368023,3.093922,3.407592,1.250121,1.788306
5,PDX156,562_17,361,100,PD,0.958840,0.104656,1.442747,2.425782,0.502635,...,2.486799,3.315665,3.105388,0.858538,0.000000,3.549519,1.824565,3.315615,0.414290,1.785053
6,PDX179,592_17,971,100,PD,0.880313,0.326937,1.200824,2.401338,0.036529,...,2.024072,3.380312,3.511453,0.423909,0.149571,3.898010,1.832906,3.473601,0.966129,1.722064
7,PDX221,584_17,819,100,PD,0.969920,0.000000,1.379578,1.651556,0.022335,...,2.442930,3.410061,3.025269,0.851106,0.362019,3.616532,2.218578,2.419985,0.710796,1.688046
8,PDX230,608_17,1055,100,CR,0.000602,0.296849,1.616845,1.628602,0.017573,...,2.132370,3.266582,2.514095,0.054267,0.162669,3.589069,2.319801,2.858703,0.237383,1.639365
9,PDX236,711_18,2215,100,PD,0.665230,0.000000,0.781327,2.569717,0.000000,...,2.127143,3.007765,3.169401,0.630687,0.000000,2.723651,2.724408,3.314286,0.618227,1.249455


In [3]:
data_columns = list(data_df.columns)[6:]

In [4]:
binary_RECISTs = []
for r in list(data_df['mRECIST']):
    if r == 'PD':
        binary_RECISTs.append(0)
    else:
        binary_RECISTs.append(1)
        
data_df['binary_RECIST'] = binary_RECISTs

In [5]:
np.unique(data_df['binary_RECIST'], return_counts=True)

(array([0, 1]), array([20,  7]))

In [6]:
#Cols E are the labels (RECIST criteria for mice), and F are the continuous values 
#(Resistance Fraction from mathematical model) to predict on

### Splitting the data into train and test sets for cross validation
We are using a stratified split to ensure consistent distribution of classes.

In [7]:
'''
# Non-stratified version
indices = np.random.permutation(len(data_df))
test_sets = [
    indices[:5],
    indices[5:10],
    indices[10:15],
    indices[15:21],
    indices[21:]
]
train_sets = [np.concatenate([indices[:i], indices[j:]]) for i, j in zip([0,5,10,15,21],[5,10,15,21,27])]

print(sorted(train_sets[1]), sorted(test_sets[1]))
'''

'\n# Non-stratified version\nindices = np.random.permutation(len(data_df))\ntest_sets = [\n    indices[:5],\n    indices[5:10],\n    indices[10:15],\n    indices[15:21],\n    indices[21:]\n]\ntrain_sets = [np.concatenate([indices[:i], indices[j:]]) for i, j in zip([0,5,10,15,21],[5,10,15,21,27])]\n\nprint(sorted(train_sets[1]), sorted(test_sets[1]))\n'

In [18]:
strat_k = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

indices = np.arange(len(binary_RECISTs))

for i, (train_idx, test_idx) in enumerate(strat_k.split(indices, binary_RECISTs), 1):
    print(train_idx, test_idx)

[ 1  2  3  5  6  7  8 10 11 12 14 15 16 17 18 20 21 23 24 25 26] [ 0  4  9 13 19 22]
[ 0  1  2  3  4  5  7  9 10 11 12 13 14 15 16 18 19 20 22 24 25] [ 6  8 17 21 23 26]
[ 0  1  2  3  4  6  8  9 10 11 12 13 15 16 17 18 19 21 22 23 25 26] [ 5  7 14 20 24]
[ 0  1  4  5  6  7  8  9 10 11 13 14 15 16 17 19 20 21 22 23 24 26] [ 2  3 12 18 25]
[ 0  2  3  4  5  6  7  8  9 12 13 14 17 18 19 20 21 22 23 24 25 26] [ 1 10 11 15 16]


# Classification of RECIST values

In [19]:
feature_importances_gbc = pd.DataFrame(index = data_columns)
feature_importances_rf = pd.DataFrame(index = data_columns)
feature_importances_lr = pd.DataFrame(index = data_columns)

'''
for n in range(5):
    train_idxs = train_sets[n]
    test_idxs = test_sets[n]
'''
for n, (train_idxs, test_idxs) in enumerate(strat_k.split(indices, binary_RECISTs), 1):
    
    X_train = data_df.loc[train_idxs][data_columns].values
    y_train = list(data_df.loc[train_idxs]['binary_RECIST'])
    
    X_test = data_df.loc[test_idxs][data_columns].values
    y_test = list(data_df.loc[test_idxs]['binary_RECIST'])
    
    print('Fold: ' + str(n), '\n')
    
    # Fitting a gradient boosting classifier and making predictions on the test set
    gbc = GradientBoostingClassifier().fit(X_train, y_train)
    y_pred_gbc = gbc.predict_proba(X_test)[:,1]
    y_pred_bin_gbc = gbc.predict(X_test)
    
    acc_gbc = accuracy_score(y_test, y_pred_bin_gbc)
    f1_gbc = f1_score(y_test, y_pred_bin_gbc)
    try:
        auc_gbc = roc_auc_score(y_test, y_pred_gbc)
    except:
        auc_gbc = 'AUC not applicable'
        
    print('Gradient Boosting:', 'Accuracy:', acc_gbc, ', F1:', f1_gbc, ', AUC:', auc_gbc)
        
    # Fitting a random forest classifier and making predictions on the test set
    rf = RandomForestClassifier().fit(X_train, y_train)
    y_pred_rf = rf.predict_proba(X_test)[:,1]
    y_pred_bin_rf = rf.predict(X_test)
    
    acc_rf = accuracy_score(y_test, y_pred_bin_rf)
    f1_rf = f1_score(y_test, y_pred_bin_rf)
    try:
        auc_rf = roc_auc_score(y_test, y_pred_rf)
    except:
        auc_rf = 'AUC not applicable'
    
    
    print('Random Forest:', 'Accuracy:', acc_rf, ', F1:', f1_rf, ', AUC:', auc_rf)
    
    # Fitting a Logistic Regression model and making predictions on the test set
    lr = LogisticRegression(max_iter=500).fit(X_train, y_train)
    y_pred_lr = lr.predict_proba(X_test)[:,1]
    y_pred_bin_lr = lr.predict(X_test)
    
    acc_lr = accuracy_score(y_test, y_pred_bin_lr)
    f1_lr = f1_score(y_test, y_pred_bin_lr)
    try:
        auc_lr = roc_auc_score(y_test, y_pred_lr)
    except:
        auc_lr = 'AUC not applicable'
    
    
    print('Logistic Regression:', 'Accuracy:', acc_lr, ', F1:', f1_lr, ', AUC:', auc_lr)
    
    # Extracting feature importances
    feature_importances_gbc['Fold ' + str(n)] = gbc.feature_importances_
    feature_importances_rf['Fold ' + str(n)] = rf.feature_importances_
    feature_importances_lr['Fold ' + str(n)] = lr.coef_[0]
    
    print('\n\n')

Fold: 1 

Gradient Boosting: Accuracy: 0.5 , F1: 0.4 , AUC: 0.5
Random Forest: Accuracy: 0.6666666666666666 , F1: 0.0 , AUC: 0.1875
Logistic Regression: Accuracy: 0.6666666666666666 , F1: 0.0 , AUC: 0.75



Fold: 2 

Gradient Boosting: Accuracy: 0.5 , F1: 0.4 , AUC: 0.625
Random Forest: Accuracy: 0.6666666666666666 , F1: 0.0 , AUC: 0.625
Logistic Regression: Accuracy: 0.5 , F1: 0.0 , AUC: 0.5



Fold: 3 

Gradient Boosting: Accuracy: 0.8 , F1: 0.0 , AUC: 0.0
Random Forest: Accuracy: 0.8 , F1: 0.0 , AUC: 1.0
Logistic Regression: Accuracy: 1.0 , F1: 1.0 , AUC: 1.0



Fold: 4 

Gradient Boosting: Accuracy: 0.8 , F1: 0.6666666666666666 , AUC: 0.875
Random Forest: Accuracy: 0.8 , F1: 0.0 , AUC: 1.0
Logistic Regression: Accuracy: 0.8 , F1: 0.0 , AUC: 0.25



Fold: 5 

Gradient Boosting: Accuracy: 1.0 , F1: 1.0 , AUC: 1.0
Random Forest: Accuracy: 0.8 , F1: 0.0 , AUC: 1.0
Logistic Regression: Accuracy: 0.6 , F1: 0.0 , AUC: 0.75





## Aggregating important features

In [20]:
feature_importances_gbc['mean_importance'] = feature_importances_gbc.mean(axis=1)
feature_importances_rf['mean_importance'] = feature_importances_rf.mean(axis=1)
feature_importances_lr['mean_weight'] = feature_importances_lr.mean(axis=1) 
# Logistic regression feature extraction works slightly differently, as weights can be positive or negative.
# For logistic regression, negative weights imply a relationship with the '0' class in binary classification
# if your input data is positive.

In [21]:
feature_importances_gbc.sort_values(by='mean_importance', ascending=False)

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,mean_importance
KIF23,0.0,0.000205,0.6067,0.784314,1.715158e-02,0.281674
DNMT1,1.0,0.000835,0.0000,0.000000,8.180708e-11,0.200167
ASNS,0.0,0.390786,0.0000,0.000000,6.655247e-02,0.091468
SRSF2,0.0,0.390464,0.0000,0.000000,0.000000e+00,0.078093
BIRC5,0.0,0.000000,0.0000,0.000000,3.463586e-01,0.069272
...,...,...,...,...,...,...
IDH1,0.0,0.000000,0.0000,0.000000,0.000000e+00,0.000000
IGF1R,0.0,0.000000,0.0000,0.000000,0.000000e+00,0.000000
IGF2,0.0,0.000000,0.0000,0.000000,0.000000e+00,0.000000
IL7R,0.0,0.000000,0.0000,0.000000,0.000000e+00,0.000000


In [22]:
feature_importances_rf.sort_values(by='mean_importance', ascending=False)

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,mean_importance
KRAS,0.000000,0.039165,0.022259,0.000000,0.047599,0.021805
KIF23,0.014632,0.006769,0.018254,0.044905,0.022521,0.021416
BIRC5,0.017812,0.026205,0.024583,0.012659,0.018491,0.019950
SMARCB1,0.045059,0.017529,0.004074,0.032479,0.000000,0.019828
ASNS,0.010917,0.007529,0.016329,0.010465,0.051398,0.019328
...,...,...,...,...,...,...
PDCD4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PBLD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PAX5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
PAPPA2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
feature_importances_lr.sort_values(by='mean_weight', ascending=False)

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,mean_weight
CTAG1B,0.155227,0.280230,0.422331,0.410267,0.394186,0.332448
ALCAM,0.176145,0.135400,0.207341,0.110831,0.127230,0.151389
CTAG2,0.253365,0.070837,0.073239,0.173859,0.161940,0.146648
PTEN,0.200457,0.005845,0.124930,0.165159,0.222162,0.143711
PRKAA2,0.154942,0.122740,0.164724,0.085900,0.159802,0.137622
...,...,...,...,...,...,...
IRS2,-0.055499,-0.235964,-0.139651,-0.174069,-0.147408,-0.150518
STMN1,-0.154497,-0.141567,-0.158995,-0.152297,-0.202972,-0.162065
ASNS,-0.173067,-0.164915,-0.139546,-0.139955,-0.196351,-0.162767
BRCA1,-0.235328,-0.127588,-0.241059,-0.185449,-0.157894,-0.189464


In [24]:
feature_importances_gbc.to_csv('./classification_outputs/gradient_boosting_feature_importances.csv')
feature_importances_rf.to_csv('./classification_outputs/random_forest_feature_importances.csv')
feature_importances_lr.to_csv('./classification_outputs/logistic_regression_feature_importances.csv')

# Regression on FR

In [40]:
from sklearn.metrics import mean_squared_error

feature_importances_gbc_regression = pd.DataFrame(index = data_columns)
feature_importances_rf_regression = pd.DataFrame(index = data_columns)
feature_importances_lr_regression = pd.DataFrame(index = data_columns)


for n, (train_idxs, test_idxs) in enumerate(strat_k.split(indices, binary_RECISTs), 1):
    
    X_train = data_df.loc[train_idxs][data_columns].values
    y_train = list(data_df.loc[train_idxs]['FR'])
    
    X_test = data_df.loc[test_idxs][data_columns].values
    y_test = list(data_df.loc[test_idxs]['FR'])
    
    print('Fold: ' + str(n), '\n')
    
    # Fitting a gradient boosting regressor and making predictions on the test set
    gbc = GradientBoostingRegressor().fit(X_train, y_train)
    y_pred_gbc = gbc.predict(X_test)
    rmse_gbc = mean_squared_error(y_test, y_pred_gbc, squared=False)
        
    print('Gradient Boosting RMSE:', rmse_gbc)
        
    # Fitting a random forest regressor and making predictions on the test set
    rf = RandomForestRegressor().fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
    
    print('Random Forest RMSE:', rmse_rf)
    
    # Fitting a Linear Regression model and making predictions on the test set
    lr = LinearRegression().fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)
    
    print('Linear Regression RMSE:', rmse_lr)
    
    # Extracting feature importances
    feature_importances_gbc_regression['Fold ' + str(n)] = gbc.feature_importances_
    feature_importances_rf_regression['Fold ' + str(n)] = rf.feature_importances_
    feature_importances_lr_regression['Fold ' + str(n)] = lr.coef_
    
    print('\n\n')

Fold: 1 

Gradient Boosting RMSE: 0.3031194465028449
Random Forest RMSE: 0.29027344679120726
Linear Regression RMSE: 0.3123230835447615



Fold: 2 

Gradient Boosting RMSE: 0.23649620810905705
Random Forest RMSE: 0.28548780375776056
Linear Regression RMSE: 0.38575968814548445



Fold: 3 

Gradient Boosting RMSE: 0.18386219923656935
Random Forest RMSE: 0.28754215184188553
Linear Regression RMSE: 0.2852446422234919



Fold: 4 

Gradient Boosting RMSE: 0.41027977029634216
Random Forest RMSE: 0.3575780859600612
Linear Regression RMSE: 0.3717796304668118



Fold: 5 

Gradient Boosting RMSE: 0.4341469940592618
Random Forest RMSE: 0.33835191777885165
Linear Regression RMSE: 0.29762767982652044





In [41]:
feature_importances_gbc_regression['mean_importance'] = feature_importances_gbc_regression.mean(axis=1)
feature_importances_rf_regression['mean_importance'] = feature_importances_rf_regression.mean(axis=1)
feature_importances_lr_regression['mean_weight'] = feature_importances_lr_regression.mean(axis=1) 

In [42]:
feature_importances_gbc_regression.sort_values(by='mean_importance', ascending=False)

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,mean_importance
WEE1,1.374183e-03,5.573674e-01,6.469104e-01,8.173350e-01,0.000561,4.047096e-01
BIRC5,4.722372e-10,7.904608e-05,1.461908e-05,1.276223e-08,0.805839,1.611866e-01
DNMT1,6.196905e-01,7.567954e-07,9.949623e-10,0.000000e+00,0.000066,1.239514e-01
PIK3R1,8.495128e-02,1.282379e-06,3.135215e-11,4.341469e-03,0.000105,1.787989e-02
EPHB2,0.000000e+00,2.724911e-07,4.282947e-09,3.018226e-09,0.080545,1.610902e-02
...,...,...,...,...,...,...
NCOA3,0.000000e+00,2.836891e-11,0.000000e+00,0.000000e+00,0.000000,5.673781e-12
DEFA1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
TET2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
SMARCA4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00


In [43]:
feature_importances_rf_regression.sort_values(by='mean_importance', ascending=False)

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,mean_importance
WEE1,9.838849e-02,1.241290e-01,1.640631e-01,3.451127e-01,1.037601e-01,1.670907e-01
BIRC5,3.403455e-02,2.621986e-02,3.340885e-02,2.387293e-02,2.829974e-01,8.010671e-02
KIF23,9.211343e-03,3.422687e-02,2.789847e-02,6.944805e-02,1.942928e-02,3.204280e-02
NCOA2,1.678693e-02,5.085262e-02,2.939848e-02,2.553380e-02,8.648112e-03,2.624399e-02
SMAD4,1.045851e-01,2.770843e-03,1.316482e-02,1.591464e-04,4.634289e-04,2.422866e-02
...,...,...,...,...,...,...
DDX43,0.000000e+00,4.804439e-08,1.981280e-05,5.895309e-08,0.000000e+00,3.983959e-06
SSX4,2.882419e-06,1.071013e-07,0.000000e+00,2.662341e-06,1.396897e-05,3.924165e-06
SSX2,0.000000e+00,5.332290e-08,4.041121e-08,1.863221e-05,8.852065e-08,3.762892e-06
FCGR2B,7.421240e-06,1.191684e-07,3.520267e-06,0.000000e+00,2.756011e-06,2.763337e-06


In [44]:
feature_importances_lr_regression.sort_values(by='mean_weight', ascending=False)

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,mean_weight
CDK6,0.023344,0.025780,0.023612,0.021492,0.026634,0.024172
BCL2,0.019763,0.019984,0.016553,0.013392,0.016125,0.017163
ASNS,0.017238,0.014240,0.010751,0.013157,0.019186,0.014915
DNMT1,0.018822,0.016006,0.015459,0.009153,0.013001,0.014488
ERCC5,0.013456,0.017352,0.013816,0.010284,0.010970,0.013175
...,...,...,...,...,...,...
ALCAM,-0.016654,-0.013775,-0.018366,-0.009181,-0.016169,-0.014829
AEBP1,-0.009432,-0.004923,-0.027738,-0.018801,-0.019934,-0.016166
PTEN,-0.015857,-0.009255,-0.013903,-0.029915,-0.025227,-0.018831
CTAG2,-0.030920,-0.015034,-0.010777,-0.022720,-0.024873,-0.020865


In [45]:
feature_importances_gbc_regression.to_csv('./regression_outputs/regression_gradient_boosting_feature_importances.csv')
feature_importances_rf_regression.to_csv('./regression_outputs/regression_random_forest_feature_importances.csv')
feature_importances_lr_regression.to_csv('./regression_outputs/regression_logistic_regression_feature_importances.csv')